In [6]:
# !pip install levenshtein
# !pip install fuzzywuzzy

  Obtaining dependency information for levenshtein from https://files.pythonhosted.org/packages/2f/04/a557197d0ba601f09fdd9ef9174b61d87706a291cd1e1578728eda3f44ab/Levenshtein-0.23.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for rapidfuzz<4.0.0,>=3.1.0 from https://files.pythonhosted.org/packages/89/49/e95e84ed1e5c3c69ea8849d06f8f475c41303533c86710b1c01a4f50d353/rapidfuzz-3.5.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/101.1 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.1 kB ? eta -:--:--
   ----------- --------------------------- 30.7/101.1 kB 660.6 kB/s eta 0:00:01
   -------------------------------------- 101.1/101.1 kB 966.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   --------- ------------------------------ 0.4/1.7 MB 8.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.2/1.7 MB 13.2 MB/s eta 0:00:01
   -----------------------------

In [7]:
# import packages for data handling/manipulation, plotting and for Approximate string matching

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [19]:
# import WHO alcohol data

drinks = pd.read_csv('WHO_data_alc.csv', low_memory=False, header=0)

In [20]:
drinks.columns

Index(['IndicatorCode', 'Indicator', 'ValueType', 'ParentLocationCode',
       'ParentLocation', 'Location type', 'SpatialDimValueCode', 'Location',
       'Period type', 'Period', 'IsLatestYear', 'Dim1 type', 'Dim1',
       'Dim1ValueCode', 'Dim2 type', 'Dim2', 'Dim2ValueCode', 'Dim3 type',
       'Dim3', 'Dim3ValueCode', 'DataSourceDimValueCode', 'DataSource',
       'FactValueNumericPrefix', 'FactValueNumeric', 'FactValueUoM',
       'FactValueNumericLowPrefix', 'FactValueNumericLow',
       'FactValueNumericHighPrefix', 'FactValueNumericHigh', 'Value',
       'FactValueTranslationID', 'FactComments', 'Language', 'DateModified'],
      dtype='object')

Location = Country

Period = year

Dim1 = Alcohol cateogory

FactValueNumeric = Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)

In [21]:
# keep important columns

df = drinks[['Location', 'Period', 'Dim1', 'FactValueNumeric', 'FactValueNumericLow','FactValueNumericHigh']]
df

,Location,Period,Dim1,FactValueNumeric,FactValueNumericLow,FactValueNumericHigh
0,Afghanistan,2019,Beer,0.00,0.0,0.0
1,Afghanistan,2019,Other alcoholic beverages,0.00,0.0,0.0
2,Afghanistan,2019,Wine,0.00,0.0,0.0
3,Algeria,2019,Other alcoholic beverages,0.00,0.0,0.0
4,Andorra,2019,Other alcoholic beverages,0.00,0.0,0.0
...,...,...,...,...,...,...
47784,United States of America,1960,Spirits,3.26,NaN,NaN
47785,United States of America,1960,Beer,3.75,NaN,NaN
47786,Austria,1960,Beer,4.70,NaN,NaN
47787,United States of America,1960,All types,7.83,NaN,NaN


In [22]:
df.drop(['Period'], axis=1).groupby('Dim1').mean().T.boxplot(figsize=(10,10))

TypeError: Could not convert BangladeshKuwaitMauritaniaSaudi ArabiaSomaliaAfghanistanLibyaIran (Islamic Republic of)YemenPakistanBhutanIndonesiaNigerSyrian Arab RepublicEgyptIraqComorosDjiboutiJordanSenegalTongaGuineaNepalTimor-LesteKiribatiOmanMoroccoChadDemocratic Republic of the CongoAlgeriaMaliMalaysiaBrunei DarussalamTajikistanMadagascarTuvaluEritreaCentral African RepublicQatarLebanonEthiopiaTürkiyeBahrainSolomon IslandsBeninPapua New GuineaAzerbaijanMaldivesTogoMozambiqueTunisiaMicronesia (Federated States of)GhanaVanuatuGuatemalaKenyaCôte d’IvoireSingaporeSlovakiaPortugalGermanyBelarusSpainHungaryIrelandPolandRomaniaAndorraLuxembourgSloveniaBulgariaFranceEstoniaAntigua and BarbudaAustriaLithuaniaCzechiaLatviaCook IslandsUnited Arab EmiratesMalawiMyanmarSamoaNamibiaNauruUzbekistanVenezuela (Bolivarian Republic of)Sri LankaGambiaFijiHondurasHaitiTurkmenistanEl SalvadorBolivia (Plurinational State of)EcuadorCosta RicaIsraelIndiaZimbabweLiberiaGuinea-BissauSierra LeoneMauritiusViet

In [31]:
ax = df.drop(['Period'], axis=1).groupby('Dim1').mean().plot(kind='bar', figsize=(15,15), title='Global Average of Alcohol Consumption, by Type', 
                                                        ylabel='Alcohol (recorded per capita, 15+) Consumption in litres of pure alcohol', xlabel='Alcohol Cateogory')

ax.tick_params(rotation=45)
ax.legend(["Countries with moderate alcohol consumption", "Countries with low alcohol consumption","Countries with high alcohol consumption"])

TypeError: Could not convert BangladeshKuwaitMauritaniaSaudi ArabiaSomaliaAfghanistanLibyaIran (Islamic Republic of)YemenPakistanBhutanIndonesiaNigerSyrian Arab RepublicEgyptIraqComorosDjiboutiJordanSenegalTongaGuineaNepalTimor-LesteKiribatiOmanMoroccoChadDemocratic Republic of the CongoAlgeriaMaliMalaysiaBrunei DarussalamTajikistanMadagascarTuvaluEritreaCentral African RepublicQatarLebanonEthiopiaTürkiyeBahrainSolomon IslandsBeninPapua New GuineaAzerbaijanMaldivesTogoMozambiqueTunisiaMicronesia (Federated States of)GhanaVanuatuGuatemalaKenyaCôte d’IvoireSingaporeSlovakiaPortugalGermanyBelarusSpainHungaryIrelandPolandRomaniaAndorraLuxembourgSloveniaBulgariaFranceEstoniaAntigua and BarbudaAustriaLithuaniaCzechiaLatviaCook IslandsUnited Arab EmiratesMalawiMyanmarSamoaNamibiaNauruUzbekistanVenezuela (Bolivarian Republic of)Sri LankaGambiaFijiHondurasHaitiTurkmenistanEl SalvadorBolivia (Plurinational State of)EcuadorCosta RicaIsraelIndiaZimbabweLiberiaGuinea-BissauSierra LeoneMauritiusViet

In [13]:
# separate the data into alcohol types

all_types = df[df['Dim1'] == 'All types'].drop(['Dim1','FactValueNumericLow', 'FactValueNumericHigh'], axis=1).rename(columns={'FactValueNumeric':'All Types Volume'})
beer = df[df['Dim1'] == 'Beer'].drop(['Dim1','FactValueNumericLow', 'FactValueNumericHigh'], axis=1).rename(columns={'FactValueNumeric':'Beer Volume'})
wine = df[df['Dim1'] == 'Wine'].drop(['Dim1','FactValueNumericLow', 'FactValueNumericHigh'], axis=1).rename(columns={'FactValueNumeric':'Wine Volume'})
spirits = df[df['Dim1'] == 'Spirits'].drop(['Dim1','FactValueNumericLow', 'FactValueNumericHigh'], axis=1).rename(columns={'FactValueNumeric':'Spirits Volume'})
other = df[df['Dim1'] == 'Other alcoholic beverages'].drop(['Dim1','FactValueNumericLow', 'FactValueNumericHigh'], axis=1).rename(columns={'FactValueNumeric':'Other Volume'})

In [14]:
all_types.groupby('Period').mean().plot(kind='line', figsize=(15,5))

TypeError: Could not convert United States of AmericaAustria to numeric

In [33]:
# import the data for the gdp per capita, growth per capita & ppp per captia

gdp_pc = pd.read_csv('gdp_per_capita.csv')
gdp_pc = gdp_pc.rename(columns={'Country Name':'Location'})

gdp_g_pc = pd.read_csv('gdp_per_capita_growth.csv')
gdp_g_pc = gdp_g_pc.rename(columns={'Country Name':'Location'})

gdp_ppp = pd.read_csv('gdp_ppp_per_capita.csv')
gdp_ppp = gdp_ppp.rename(columns={'Country Name':'Location'})

In [34]:
# approximate string matching fucntion, to find differences in the country names between the datasets

def NameFinder(name):
    alc_names = df['Location'].drop_duplicates().to_list()
    gdp_names = gdp_pc['Location'].drop_duplicates().to_list()
    alc_name = process.extract(name, alc_names, limit=1)
    gdp_name = process.extract(name, gdp_names, limit=1)

    print('Country Name in GDP Dataset', gdp_name, 'Country Name in Alc Dataset', alc_name)
    return gdp_name,alc_name

In [27]:
# returns the number of locations that do not match, the list of the names that do not match

gdp_locs = gdp_pc['Location'].drop_duplicates().to_list()
alc_locs = df['Location'].drop_duplicates().to_list()
len(gdp_locs), len(alc_locs)

closest_match=[]
for i in gdp_locs:
    fm = process.extract(i, alc_locs, limit=1)
    closest_match.append([i, fm[0]])

non_match=[]
for i in closest_match:
    if i[1][1] < 100:
        non_match.append(i)

print(len(non_match))

91


In [28]:
# countries with non-matching names - Bahamas, Bolivia, Ivory Coast, Egypt, UK, Gambia, Iran, St. Lucia, Moldova, Macedonia, Tanzania,  USA, Venexuela, Vietnam, Yemen

g, a = NameFinder('Yemen')

Country Name in GDP Dataset [('Yemen', 100)] Country Name in Alc Dataset [('Yemen', 100)]


In [29]:
# replace names is the gdp datasets that don't match with those in the alcohol data

gdp_pc.loc[gdp_pc['Location'] == g[0][0], 'Location'] = a[0][0]

gdp_g_pc.loc[gdp_g_pc['Location'] == g[0][0], 'Location'] = a[0][0]

gdp_ppp.loc[gdp_ppp['Location'] == g[0][0], 'Location'] = a[0][0]

In [15]:
# # save the corrected datasets

# gdp_pc.to_csv('gdp_per_capita.csv')
# gdp_g_pc.to_csv('gdp_per_capita_growth.csv')
# gdp_ppp.to_csv('gdp_ppp_per_capita.csv')

In [30]:
# function to find the gdp of a desired country

def CountryGDP(country):
    country_gdp = gdp_pc[gdp_pc['Country Name'] == country].T.drop(['Country Name', 'Code', 'Unnamed: 65'])
    country_gdp['Year'] = country_gdp.index
    country_gdp['Year'] = country_gdp['Year'].astype(int)
    country_gdp.columns = ['GDP', 'Year']
    country_gdp = country_gdp.reset_index()
    country_gdp = country_gdp.drop(['index'], axis=1)
    return country_gdp

In [17]:
# function to find the total alcohol consumption of a desired country

def CountryConspumtion(country):
    country_cons = df[df['Location'] == country][['Period', 'Dim1','FactValueNumeric']].reset_index().drop(['index'], axis=1)
    country_cons.columns = ['Year', 'Alcohol Type','alc_cons_litre']
    country_cons = country_cons.sort_values('Year')
    return country_cons

In [18]:
# function to plot the gdp of a country vs the consumption on the same axis

def TwoAxisPlot(df1, df2, country):
    x1 = np.arange(0, len(df1), step=1)
    y1 = df1['GDP'].to_list()
    x2 = np.arange(0, len(df2), step=1)
    y2 = df2['alc_cons_litre'].to_list()
    years = df2['Year'].to_list()

    fig, ax = plt.subplots(figsize=(16,8))

    color = 'tab:red'
    ax.set_ylabel('GDP', color=color)
    ax.plot(x1, y1, color=color, label='GDP')
    ax.tick_params(axis='y', labelcolor=color)
    ax2 = ax.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Alcohol Consumption', color=color)
    ax2.plot(x2, y2, color=color, label='Alcohol Consumption')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()

    ax.set_xticks(np.arange(len(df2)))
    ax.set_xticklabels(years)
    ax.tick_params(labelrotation=45)
    ax.set_title(f'Normalised GDP vs Total Alcohol Consumption, {country}')
    return

In [37]:
# fucntion to collate the alcohol data, manipulate the GDP data into a usable format and merge the two
        # fucntio merges the alchol dataframes, selects the gdp data where location names match, transposes the gdp data and renames the columns, resets index
        # The create a dataframe for each country in the gdp dataset before concatting them all together - done so that the alchol and gdp data are in the same format so that they can be merged
        # the gdp dataframe and the alcholo dataframe are then merged together, dropping and NaN values

def DataSorter(gdp_df):
    merge = all_types.merge(beer).merge(wine).merge(spirits).merge(other).sort_values(by=['Location', 'Period']).reset_index().drop(['index'], axis=1)
    gdp_filt = gdp_df.loc[gdp_df['Location'].isin(merge['Location'])].drop(['2020','Unnamed: 0','Unnamed: 65'], axis=1)
    gdp_T = gdp_filt.T
    gdp_T = gdp_T.rename(columns=gdp_T.iloc[0]).iloc[1:,:].reset_index().rename(columns={'index': 'Period'})
    years = gdp_T.Period.to_list()
    locs = gdp_T.columns.to_list()
    pop = locs.pop(0)

    df_list=[]
    for i in locs:
        df_l = pd.DataFrame(list(zip(years, gdp_T[i].to_list())), columns=['Period', 'GDP'])
        df_l['Location'] = i
        df_list.append(df_l)

    print('No. of seperate DFs created equal to length of countries (from above)',len(df_list))
    gdp_concat = pd.concat(df_list)
    gdp_concat['Period'] = gdp_concat['Period'].astype(int)
    final_merge = merge.merge(gdp_concat)
    final_merge = final_merge.dropna()
    return final_merge

In [38]:
gdp_pc_merge = DataSorter(gdp_pc)
gdp_g_pc_merge = DataSorter(gdp_g_pc)
gdp_ppp_merge = DataSorter(gdp_ppp)

No. of seperate DFs created equal to length of countries (from above) 175
No. of seperate DFs created equal to length of countries (from above) 175
No. of seperate DFs created equal to length of countries (from above) 175


In [21]:
# gdp_pc_merge.to_csv('merged_gdp_pc.csv')
# gdp_g_pc_merge.to_csv('merged_gdp_g_pc.csv')
# gdp_ppp_merge.to_csv('merged_gdp_ppp_pc.csv')

In [22]:
    # summary stats

gdp_pc_merge.drop(columns=['Period']).describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,6530.000000,6530.000000,6530.000000,6530.000000,6530.000000,6530.000000
mean,4.817186,1.702672,0.932966,1.486108,0.712724,8066.087003
std,3.984066,1.741868,1.835191,1.750167,1.688134,14288.449750
min,0.000000,0.000000,0.000000,0.000000,0.000000,22.795345
25%,1.350000,0.300000,0.012000,0.100000,0.000000,651.582128
50%,4.015000,1.090000,0.130000,0.940000,0.010000,2166.974843
75%,7.550000,2.700000,0.970000,2.287500,0.430000,7779.738401
max,19.890000,9.960000,18.740000,14.440000,13.060000,123514.196686


In [24]:
gdp_g_pc_merge.drop(columns=['Period']).describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000
mean,4.823227,1.707050,0.942716,1.494695,0.696540,1.917651
std,3.964700,1.711774,1.847258,1.760528,1.615174,5.585587
min,0.000000,0.000000,0.000000,0.000000,0.000000,-64.992369
25%,1.390000,0.310000,0.016000,0.100000,0.000000,-0.245106
50%,4.010000,1.130000,0.130000,0.950000,0.010000,2.055111
75%,7.520000,2.720000,0.980000,2.290000,0.440000,4.377587
max,19.890000,9.960000,18.740000,14.440000,13.060000,121.779472


In [26]:
gdp_ppp_merge.drop(columns=['Period']).describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000
mean,4.788391,1.788711,0.950662,1.496167,0.570155,14355.106886
std,3.925423,1.712766,1.571085,1.721181,1.517723,18071.340379
min,0.000000,0.000000,0.000000,0.000000,0.000000,285.404911
25%,1.260000,0.330000,0.020000,0.120000,0.000000,2675.593695
50%,3.990000,1.230000,0.160000,0.970000,0.010000,7393.298339
75%,7.810000,2.940000,1.200000,2.250000,0.270000,18199.383029
max,17.870000,9.960000,10.010000,14.440000,13.060000,141634.703825


In [53]:
q75 = gdp_pc_merge.drop(columns=['Period'])['All Types Volume'].quantile(q=0.75)
gdp_pc_merge.drop(columns=['Period'])[gdp_pc_merge.drop(columns=['Period'])['All Types Volume'].ge(q75)].describe()


,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,1635.000000,1635.000000,1635.000000,1635.000000,1635.000000,1635.000000
mean,10.488856,3.633578,2.807269,2.892102,1.181654,17985.663425
std,2.235514,1.892699,2.729148,2.245129,2.475645,19613.424564
min,7.550000,0.000000,0.000000,0.000000,0.000000,40.944182
25%,8.680000,2.320000,0.840000,1.470000,0.000000,3336.699770
50%,10.070000,3.600000,2.160000,2.330000,0.150000,11175.816604
75%,11.725000,4.655000,3.920000,3.855000,0.825000,26806.329875
max,19.890000,9.960000,18.740000,14.440000,13.060000,123514.196686


In [57]:
q25 = gdp_pc_merge.drop(columns=['Period'])['All Types Volume'].quantile(q=0.25)
gdp_pc_merge.drop(columns=['Period'])[gdp_pc_merge.drop(columns=['Period'])['All Types Volume'].le(q25)].describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,1645.000000,1645.000000,1645.000000,1645.000000,1645.000000,1645.000000
mean,0.491986,0.237309,0.064110,0.143507,0.051621,3806.722982
std,0.411312,0.263911,0.127093,0.223832,0.162913,9060.057416
min,0.000000,0.000000,0.000000,0.000000,0.000000,22.795345
25%,0.100000,0.020000,0.000000,0.010000,0.000000,411.859439
50%,0.430000,0.150000,0.010000,0.047000,0.000000,942.518916
75%,0.810000,0.360000,0.070000,0.180000,0.002000,2791.810766
max,1.350000,1.600000,1.120000,1.300000,1.550000,85075.986537


In [70]:
gdp_pc_merge_g25 = gdp_pc_merge.drop(columns=['Period'])[gdp_pc_merge.drop(columns=['Period'])['All Types Volume'].ge(q25)]
gdp_pc_merge_g25l75 = gdp_pc_merge_g25[gdp_pc_merge.drop(columns=['Period'])['All Types Volume'].le(q75)]
gdp_pc_merge_g25l75.describe()

C:\Users\janno\AppData\Local\Temp\ipykernel_12560\255970150.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gdp_pc_merge_g25l75 = gdp_pc_merge_g25[gdp_pc_merge.drop(columns=['Period'])['All Types Volume'].le(q75)]


,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,3270.000000,3270.000000,3270.000000,3270.000000,3270.000000,3270.000000
mean,4.143557,1.469570,0.430078,1.453508,0.809786,5217.193538
std,1.797286,1.113976,0.782960,1.299019,1.512728,10395.128960
min,1.350000,0.000000,0.000000,0.000000,0.000000,37.488839
25%,2.610000,0.530000,0.028250,0.342500,0.000000,588.974108
50%,4.005000,1.240000,0.110000,1.200000,0.030000,1621.360120
75%,5.647500,2.160000,0.397500,2.250000,1.070000,4712.503555
max,7.550000,6.020000,5.820000,6.700000,7.540000,102913.450844


In [72]:
q75 = gdp_g_pc_merge.drop(columns=['Period'])['All Types Volume'].quantile(q=0.75)
gdp_g_pc_merge.drop(columns=['Period'])[gdp_g_pc_merge.drop(columns=['Period'])['All Types Volume'].ge(q75)].describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,1590.000000,1590.000000,1590.000000,1590.000000,1590.000000,1590.000000
mean,10.458962,3.609132,2.844934,2.939564,1.096512,1.977877
std,2.240506,1.812573,2.732131,2.250311,2.292931,4.526741
min,7.520000,0.000000,0.000000,0.000000,0.000000,-47.503237
25%,8.650000,2.330000,0.882500,1.490000,0.000000,0.191038
50%,10.010000,3.580000,2.225000,2.355000,0.150000,2.028640
75%,11.660000,4.597500,3.947500,3.937500,0.810000,4.149441
max,19.890000,9.960000,18.740000,14.440000,13.060000,37.535461


In [73]:
q25 = gdp_g_pc_merge.drop(columns=['Period'])['All Types Volume'].quantile(q=0.25)
gdp_g_pc_merge.drop(columns=['Period'])[gdp_g_pc_merge.drop(columns=['Period'])['All Types Volume'].le(q25)].describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,1591.000000,1591.000000,1591.000000,1591.000000,1591.000000,1591.000000
mean,0.503468,0.242051,0.065780,0.143192,0.057181,1.606586
std,0.417804,0.264312,0.128483,0.222412,0.175436,6.985566
min,0.000000,0.000000,0.000000,0.000000,0.000000,-64.992369
25%,0.120000,0.030000,0.000000,0.010000,0.000000,-0.852169
50%,0.440000,0.160000,0.010000,0.049000,0.000000,1.808253
75%,0.820000,0.370000,0.070000,0.170000,0.002000,4.256556
max,1.390000,1.600000,1.120000,1.300000,1.550000,121.779472


In [75]:
gdp_g_pc_merge_g25 = gdp_g_pc_merge.drop(columns=['Period'])[gdp_g_pc_merge.drop(columns=['Period'])['All Types Volume'].ge(q25)]
gdp_g_pc_merge_g25l75 = gdp_g_pc_merge_g25[gdp_g_pc_merge.drop(columns=['Period'])['All Types Volume'].le(q75)]
gdp_g_pc_merge_g25l75.describe()

C:\Users\janno\AppData\Local\Temp\ipykernel_12560\1681360581.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gdp_g_pc_merge_g25l75 = gdp_g_pc_merge_g25[gdp_g_pc_merge.drop(columns=['Period'])['All Types Volume'].le(q75)]


,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,3178.000000,3178.000000,3178.000000,3178.000000,3178.000000,3178.000000
mean,4.163096,1.485234,0.428396,1.445825,0.821181,2.041396
std,1.778998,1.116797,0.786175,1.295093,1.515349,5.256776
min,1.390000,0.000000,0.000000,0.000000,0.000000,-43.332492
25%,2.650000,0.540000,0.029000,0.340000,0.000000,-0.141142
50%,4.005000,1.270000,0.110000,1.195000,0.030000,2.198046
75%,5.670000,2.180000,0.380000,2.240000,1.090000,4.507785
max,7.520000,6.020000,5.820000,6.700000,7.260000,56.788936


In [76]:
q75 = gdp_ppp_merge.drop(columns=['Period'])['All Types Volume'].quantile(q=0.75)
gdp_ppp_merge.drop(columns=['Period'])[gdp_ppp_merge.drop(columns=['Period'])['All Types Volume'].ge(q75)].describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000
mean,10.350060,3.715801,2.653175,3.014487,0.981306,24219.625812
std,1.817922,1.614204,2.023723,2.107033,2.230445,17240.529778
min,7.810000,0.100000,0.000000,0.000000,0.000000,355.632434
25%,8.905000,2.535000,0.990000,1.540000,0.000000,11246.415292
50%,10.010000,3.800000,2.410000,2.390000,0.170000,21270.266127
75%,11.380000,4.700000,3.870000,4.070000,0.720000,33520.341212
max,17.870000,9.960000,10.010000,14.440000,13.060000,117341.915064


In [77]:
q25 = gdp_ppp_merge.drop(columns=['Period'])['All Types Volume'].quantile(q=0.25)
gdp_ppp_merge.drop(columns=['Period'])[gdp_ppp_merge.drop(columns=['Period'])['All Types Volume'].le(q25)].describe()

,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000
mean,0.446702,0.242108,0.050831,0.117429,0.037616,11347.404939
std,0.382892,0.251818,0.094050,0.181500,0.119394,21286.958869
min,0.000000,0.000000,0.000000,0.000000,0.000000,285.404911
25%,0.090000,0.040000,0.000000,0.010000,0.000000,1596.232027
50%,0.365000,0.160000,0.010000,0.040000,0.000000,2950.212873
75%,0.710000,0.380000,0.060000,0.150000,0.002000,9518.113464
max,1.260000,1.150000,0.890000,1.210000,0.910000,141634.703825


In [78]:
gdp_ppp_merge_g25 = gdp_ppp_merge.drop(columns=['Period'])[gdp_ppp_merge.drop(columns=['Period'])['All Types Volume'].ge(q25)]
gdp_ppp_merge_g25l75 = gdp_ppp_merge_g25[gdp_ppp_merge.drop(columns=['Period'])['All Types Volume'].le(q75)]
gdp_ppp_merge_g25l75.describe()

C:\Users\janno\AppData\Local\Temp\ipykernel_12560\7797818.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gdp_ppp_merge_g25l75 = gdp_ppp_merge_g25[gdp_ppp_merge.drop(columns=['Period'])['All Types Volume'].le(q75)]


,All Types Volume,Beer Volume,Wine Volume,Spirits Volume,Other Volume,GDP
count,2338.000000,2338.000000,2338.000000,2338.000000,2338.000000,2338.000000
mean,4.167233,1.592179,0.550841,1.419871,0.630803,10878.799170
std,1.949626,1.196240,0.938261,1.253696,1.372313,14552.279096
min,1.260000,0.000000,0.000000,0.000000,0.000000,527.080875
25%,2.440000,0.540000,0.040000,0.390000,0.000000,2767.492535
50%,3.985000,1.370000,0.150000,1.140000,0.021000,6306.629395
75%,5.817500,2.390000,0.540000,2.140000,0.340000,12205.522120
max,7.810000,6.020000,5.320000,6.560000,7.540000,102573.401768
